In [1]:
from summarize_from_feedback import sample
from summarize_from_feedback.utils import experiment_helpers as utils
from summarize_from_feedback.query_response_model import ModelSpec
from summarize_from_feedback.tasks import TaskHParams, TaskQueryHParams, TaskResponseHParams
import os

os.environ["JOB_NAME"] = "local" #For MPI-dependent logging in sample.main

HParams = sample.HParams(
     model_spec = ModelSpec(
         load_path="https://openaipublic.blob.core.windows.net/summarize-from-feedback/models/sup4_ppo_rm4",
         short_name="sup4_ppo_rm4",
     ),
     task = TaskHParams(
         query = TaskQueryHParams(
             format_str = "SUBREDDIT: r/{subreddit}\n\nTITLE: {title}\n\nPOST: {post}\n\nTL;DR:",
             dataset = "tldr_3_filtered",
             length =  512,
             truncate_text = "\n",
             truncate_field = "post",
             pad_side = "left"   
         ),
         response = TaskResponseHParams(
             length = 48,
             truncate_token = 50256,  # endoftext
             ref_format_str = " {reference}",  # add a leading space 
         )
     ),
     query_dataset_split= "valid",
     num_queries = 1
    )

#print(utils.sup4_ppo_rm4())
sample.main(H = HParams)

copying https://openaipublic.blob.core.windows.net/summarize-from-feedback/models/sup4_ppo_rm4/done-sentinel /tmp/bf-dir-cache/az/openaipublic/summarize-from-feedback/models/sup4_ppo_rm4/done-sentinel
copying https://openaipublic.blob.core.windows.net/summarize-from-feedback/models/sup4_ppo_rm4/info.json /tmp/bf-dir-cache/az/openaipublic/summarize-from-feedback/models/sup4_ppo_rm4/info.json
copying https://openaipublic.blob.core.windows.net/summarize-from-feedback/models/sup4_ppo_rm4/checkpoint/final_layer_norm_shard_000.pkl /tmp/bf-dir-cache/az/openaipublic/summarize-from-feedback/models/sup4_ppo_rm4/checkpoint/final_layer_norm_shard_000.pkl
copying https://openaipublic.blob.core.windows.net/summarize-from-feedback/models/sup4_ppo_rm4/checkpoint/input_embeddings_shard_000.pkl /tmp/bf-dir-cache/az/openaipublic/summarize-from-feedback/models/sup4_ppo_rm4/checkpoint/input_embeddings_shard_000.pkl
copying https://openaipublic.blob.core.windows.net/summarize-from-feedback/models/sup4_ppo_r

{'output_path': '/tmp/jobs/local/results'}

In [ ]:
%run sample.py test test-sample